# Estimate depth using Marigold model

Code based on: https://colab.research.google.com/drive/12G8reD13DdpMie5ZQlaFNo2WCGeNUH-u?usp=sharing#scrollTo=3Bk37mbdhQCO

---

# 🛠️ 1 Setup

In [ ]:
# @title 💻 1.1 Check CUDA and Python Version (tested with CUDA 12.2 + Python 3.10.12)

# This notebook is supposed to be connected to NVIDIA drivers with CUDA.
# If this doesn't load check that GPU is selected as hardware accelerator under Edit -> Notebook settings.
!nvidia-smi
!python -V

/bin/bash: line 1: nvidia-smi: command not found
Python 3.10.12


In [ ]:
# Access Google Drive data
from google.colab import drive
drive.mount('/content/drive')

# SCANNET_SCENE_DIR = '/content/drive/MyDrive/3d-machine-learning/scannet/scene0710_00'
SCANNET_SCENE_DIR = '/content/drive/MyDrive/ML43D/scannetpp/7eac902fd5'

Mounted at /content/drive


In [ ]:
# # prompt: Save all files to csv

# import os
# import csv

# def save_files_to_csv(directory, csv_filename):
#   with open(csv_filename, 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     #writer.writerow(['Filename'])  # Header row

#     for filename in os.listdir(directory):
#       writer.writerow([filename])

# # Example usage:
# save_files_to_csv(os.path.join(SCANNET_SCENE_DIR, 'color'), 'train_set.csv')

In [ ]:
# @title 1.2 👆 Select Checkpoint
!pip install ipywidgets==7.7.1 --quiet

from IPython.display import display
import ipywidgets as widgets
import shutil

ckpt_dic = {
    "Original (higher quality)": "prs-eth/marigold-v1-0",
    "LCM (faster)": "prs-eth/marigold-lcm-v1-0",
}

ckpt_name = 'Original (higher quality)'  # Use higher quality model
ckpt_path = ckpt_dic[ckpt_name]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00


## 1.3 🏗️ Installation and Initialization

We provide two ways to run the pipeline. You can choose either ONE to run:

### 🅱 1.3-A Using code from the [full repository](https://github.com/prs-eth/marigold)

In [ ]:
# @title 📦 i. Clone repository

%%shell
cd /content

if [ -d "Marigold" ]; then
    cd Marigold
    git pull
else
    git clone https://github.com/prs-eth/Marigold.git
    cd Marigold
fi

Cloning into 'Marigold'...
remote: Enumerating objects: 616, done.
remote: Counting objects: 100% (327/327), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 616 (delta 235), reused 244 (delta 174), pack-reused 289
Receiving objects: 100% (616/616), 5.70 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (351/351), done.


In [ ]:
# @title 🔨 ii. Install dependencies (might take minutes)
%%shell

cd /content/Marigold

# pip install -r requirements.txt --upgrade  --quiet
pip install accelerate matplotlib scipy torch transformers --quiet
pip install diffusers==0.25.0 --quiet

# for progress bar
pip install ipywidgets==7.7.1 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.3 MB/s eta 0:00:00


In [ ]:
# @title 🔧 iii. Setup directories

import os

# Directories
repo_dir = "/content/Marigold"
input_dir = os.path.join(repo_dir, "input")
output_dir = os.path.join(repo_dir, "output")
output_dir_color = os.path.join(output_dir, "depth_colored")
output_dir_tif = os.path.join(output_dir, "depth_bw")
output_dir_npy = os.path.join(output_dir, "depth_npy")

os.chdir(repo_dir)



In [ ]:
# @title 🏇 iii. Initialize pipeline

# change model cache directory
!export HF_HOME=$(pwd)/checkpoint

from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained(
    ckpt_path,
    custom_pipeline="marigold_depth_estimation"
)

pipe = pipe.to("cuda")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  wa

model_index.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:671: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:677: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:671: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:677: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The config attributes {'scale_invariant': True, 'shift_invariant': True, 'default_denoising_steps': 10, 'default_processing_resolution': 768} were passed to MarigoldPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'scale_invariant': True, 'shift_invariant': True, 'default_denoising_steps': 10, 'default_processing_resolution': 768} are not expected by MarigoldPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# @title 🔨 i. Install dependencies

!pip install --upgrade "diffusers>=0.25.0" --quiet

# !pip install ipywidgets==7.7.1 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:00


In [ ]:
# @title 🔧 ii. Setup directories
import os


# Directories
repo_dir = "/content/Marigold"
input_dir = os.path.join(repo_dir, "input")
output_dir = os.path.join(repo_dir, "output")
output_dir_color = os.path.join(output_dir, "depth_colored")
output_dir_tif = os.path.join(output_dir, "depth_bw")
output_dir_npy = os.path.join(output_dir, "depth_npy")


os.makedirs(repo_dir, exist_ok=True)
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

os.chdir(repo_dir)

In [ ]:
# @title 🏇 iii. Initialize pipeline

# change model cache directory
!export HF_HOME=$(pwd)/checkpoint

from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained(
    ckpt_path,
    custom_pipeline="marigold_depth_estimation"
)

pipe = pipe.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:671: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:677: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The config attributes {'scale_invariant': True, 'shift_invariant': True, 'default_denoising_steps': 10, 'default_processing_resolution': 768} were passed to MarigoldPipeline, but are not expected and will 

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

## 🚀 2 Inference

In [ ]:
# @title 🖼️ 2.2.2 Prepare input data: Upload input images to "input/" folder or use our sample images (run this cell to see buttons)

from IPython.display import display
import ipywidgets as widgets
import shutil

from google.colab import files


In [ ]:
%matplotlib inline

# @title 2.2.3 Display input images

from glob import glob
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# @title ▶️ 2.2.4 Run inference (default settings are for LCM)
denoising_steps = 50
ensemble_size = 10
processing_res = 768
match_input_res = True
use_seed = False
seed = 2024

if not use_seed:
    seed = None

import logging
import numpy as np
import torch
from tqdm.auto import tqdm
from glob import glob
from PIL import Image

def run_inference(input_dir, output_dir, out_dir_uncertainty, output_dir_vis,
                  denoising_steps, ensemble_size, processing_res, match_input_res):
  # Create output folders
  os.makedirs(output_dir, exist_ok=True)
  os.makedirs(out_dir_uncertainty, exist_ok=True)
  os.makedirs(output_dir_vis, exist_ok=True)

  # Image list
  EXTENSION_LIST = [".jpg", ".jpeg", ".png"]
  rgb_filename_list = glob(os.path.join(input_dir, "*"))
  rgb_filename_list = [
      f for f in rgb_filename_list if os.path.splitext(f)[1].lower() in EXTENSION_LIST
  ]
  rgb_filename_list = sorted(rgb_filename_list)
  # Run Inference
  with torch.no_grad():
      for rgb_path in tqdm(rgb_filename_list, desc=f"Estimating depth", leave=True):
          # Read input image
          input_image = Image.open(rgb_path)

          # Predict depth
          pipeline_output = pipe(
              input_image,
              denoising_steps=denoising_steps,     # optional
              ensemble_size=ensemble_size,       # optional
              processing_res=processing_res,     # optional
              match_input_res=match_input_res,   # optional
              batch_size=0,           # optional
              color_map="Spectral",   # optional
              show_progress_bar=True, # optional
            #   seed=seed,              # optional
          )

          depth_pred: np.ndarray = pipeline_output.depth_np
          depth_colored: Image.Image = pipeline_output.depth_colored
          uncertainty: torch.tensor = pipeline_output.uncertainty
          uncertainty = uncertainty.cpu().numpy()
          print(f'Max Uncertainty: {uncertainty.max()}')
          print(f'Min Uncertainty: {uncertainty.min()}')
          # Save as npy
          rgb_name_base = os.path.splitext(os.path.basename(rgb_path))[0]
          pred_name_base = rgb_name_base
          npy_save_path = os.path.join(output_dir, f"{pred_name_base}.npy")
          if os.path.exists(npy_save_path):
              logging.warning(f"Existing file: '{npy_save_path}' will be overwritten")
          np.save(npy_save_path, depth_pred)

          # Save as 16-bit uint png
          depth_to_save = (depth_pred * 65535.0).astype(np.uint16)
          png_save_path = os.path.join(output_dir_vis, f"{pred_name_base}_gray.png")
          if os.path.exists(png_save_path):
              logging.warning(f"Existing file: '{png_save_path}' will be overwritten")
          Image.fromarray(depth_to_save).save(png_save_path, mode="I;16")

          # Colorize
          colored_save_path = os.path.join(
              output_dir_vis, f"{pred_name_base}_colored.png"
          )
          if os.path.exists(colored_save_path):
              logging.warning(f"Existing file: '{colored_save_path}' will be overwritten")
          depth_colored.save(colored_save_path)

          # Save uncertainty
          npy_save_path = os.path.join(out_dir_uncertainty, f"{pred_name_base}_uncertainty.npy")
          if os.path.exists(npy_save_path):
              logging.warning(f"Existing file: '{npy_save_path}' will be overwritten")
          np.save(npy_save_path, uncertainty)

          # Save uncertainty as 16-bit uint png
          uuncertainty_to_save = (((uncertainty - uncertainty.min()) / (uncertainty.max() - uncertainty.min())) * 65535.0).astype(np.uint16)
          png_save_path = os.path.join(output_dir_vis, f"{pred_name_base}_uncertainty_gray.png")
          if os.path.exists(png_save_path):
              logging.warning(f"Existing file: '{png_save_path}' will be overwritten")
          Image.fromarray(uuncertainty_to_save).save(png_save_path, mode="I;16")
  return

In [ ]:
input_dir = os.path.join(SCANNET_SCENE_DIR, 'color')
out_dir = os.path.join(SCANNET_SCENE_DIR, 'depth_MG')
out_dir_uncertainty = os.path.join(SCANNET_SCENE_DIR, 'uncertainty_MG')
out_dir_vis = os.path.join(SCANNET_SCENE_DIR, 'depth_MG_visualization')
run_inference(input_dir, out_dir, out_dir_uncertainty, out_dir_vis, denoising_steps, ensemble_size, processing_res, match_input_res)

Estimating depth:   0%|          | 0/25 [00:00<?, ?it/s]

  Inference batches:   0%|          | 0/5 [00:00<?, ?it/s]

    Diffusion denoising:   0%|          | 0/50 [00:00<?, ?it/s]

    Diffusion denoising:   0%|          | 0/50 [00:00<?, ?it/s]

    Diffusion denoising:   0%|          | 0/50 [00:00<?, ?it/s]

...

    Diffusion denoising:   0%|          | 0/50 [00:00<?, ?it/s]

    Diffusion denoising:   0%|          | 0/50 [00:00<?, ?it/s]

    Diffusion denoising:   0%|          | 0/50 [00:00<?, ?it/s]

    Diffusion denoising:   0%|          | 0/50 [00:00<?, ?it/s]

Max Uncertainty: 0.10884536802768707
Min Uncertainty: 0.0
